# Entity Class: Introduction

`Entity` is the base class of all other classes defined in IGLSynth. 
`Entity` class definition does the following:


* assigns a `name` to all entity objects; 
* ensures no duplicate objects are created with same `name`;
* defines the `hash` function;
* defines when two `Entity` objects are equal;
* defines default serialization of `Entity` objects;
* assigns a `logging.Logger` object to entity for recording internal events.

The remainder of tutorial elaborates on these concepts.

In [1]:
import sys
sys.path.insert(0, '/home/abhibp1993/Research/iglsynth')

In [2]:
import iglsynth.util as util

## Instantiate Entity

It is **not** recommended to directly instantiate `Entity` objects for application, although it is possible. 
The following code is only to showcase the generic methods of instantiation available to all IGLSynth objects. 

Let us start with the easiest way of instanting an `Entity`. 

In [3]:
# Create an entity with no instantiation parameters
e = util.Entity()
e

Entity(name=b38ac5f8-84e0-4167-96a8-3992f3f83e0e)

Observe that the `e1` has been assigned a name automatically. This name is generated using `uuid.uuid4()` function. See [Wikipedia](https://en.wikipedia.org/wiki/Universally_unique_identifier) for more information on uuid.

Next, we instantiate `Entity` by providing a custom name. This name can be any `<hashable>` python object.
For simplicity, we recommend this to be one of `string, int, float, tuple` or `list`.

In [4]:
# Entity with custom name
e = util.Entity(name="MyEntity")
e

Entity(name=MyEntity)

In [5]:
# Another entity with custom name
e = util.Entity(name=(10, 20))
e

Entity(name=(10, 20))

In [6]:
# Unhashable names are not allowed
try:
    e = util.Entity(name={10, 20})
except AssertionError as err:
    print(err)

Finally, it is possible to pass keyword arguments to `Entity` constructor to initialize the object with attributes.
This will come handy while reconstructing the object during deserialization. 

In [7]:
# Explicitly provide keyword arguments
e = util.Entity(name="keywords!", addition=30)
print(e)
e.addition

Entity(name=keywords!)


30

In [8]:
# Provide keyword arguments by unpacking the dictionary
e = util.Entity(**{"name": "unpacking", "coordinate": (10, 20)})
print(e)
e.coordinate

Entity(name=unpacking)


(10, 20)

## Duplication Avoidance

`Entity` construction ensures that no two entities of same (sub-)class have same names. Whenever an attempt is made to instantiate a duplicate object, the existing object is returned instead of creating a new one. 

It is important to understand how `Entity` class ensures uniqueness. During construction, `Entity` constructs a unique name of object as `<module_name>:<class_name>:<name>`. If there exists an object with this name previously instantiated then it is returned. Thus, it is possible to have two objects of, say, `Entity` subclasses `Vertex` and `Edge` with the same name, say `helloEntity`. But, it is not possible to have two `Vertex` objects with name `helloEntity`. 

Let us see an example.

In [9]:
class Vertex(util.Entity):
    pass

class Edge(util.Entity):
    pass


v1 = Vertex(name="helloEntity")
v2 = Vertex(name="helloEntity")
e1 = Edge(name="helloEntity")

print(v1, id(v1))
print(v2, id(v2))
print(e1, id(e1))

Vertex(name=helloEntity) 139757657740400
Vertex(name=helloEntity) 139757657740400
Edge(name=helloEntity) 139757657740120


Note that the two vertices `v1, v2` have the same `id`'s which imply that they are same objects in the memory. Recall that `id` function returns the memory address of the python object. 

On the contrary, `e1` has a different id than `v1` and `v2` implying that it is a different object, but with the same name. 

## Hashing and Equality

All `Entity` objects are hashable. 
The default hash value of an `Entity` is given by `hash(name)`. 

Also, two `Entity` objects are said to be equal if they have the same `name`. 

<div class="alert alert-block alert-warning">
<b>Example:</b> Use yellow boxes for examples that are not inside code cells, or use for mathematical formulas if needed. Typically also used to display warning messages.
</div>

In [10]:
e1 = util.Entity(name="1")
e2 = util.Entity(name="2")
e3 = util.Entity(name="2")

print(f"Is hash(e1) equal to hash(e1.name): {hash(e1) == hash(e1.name)}")
print(f"Is hash(e2) equal to hash(e2.name): {hash(e2) == hash(e2.name)}")
print(f"Is hash(e3) equal to hash(e3.name): {hash(e3) == hash(e3.name)}")
print()
print(f"Is e1 equal to e2: {e1 == e2}")
print(f"Is e2 equal to e3: {e2 == e3}")
print(f"Is e3 equal to e1: {e3 == e1}")

Is hash(e1) equal to hash(e1.name): True
Is hash(e2) equal to hash(e2.name): True
Is hash(e3) equal to hash(e3.name): True

Is e1 equal to e2: False
Is e2 equal to e3: True
Is e3 equal to e1: False


## Serialization of Entity

A `serialize` function is defined for `Entity` objects which returns a dictionary of attributes to their values in the object. This dictionary can later be used by functions in `readwrite` package to write to a file. 

The `serialize` function takes in a parameter `ignores` that is a list of attributes which should not be serialized. An example of an ignore is `logger`; which generally should not be saved or communicated. 

In [11]:
e1 = util.Entity(name="Entity", coordinate=(10, 20), other="ignore me")
print("Without ignore: ", e1.serialize())
print("With ignore: ", e1.serialize(ignores=["other"]))

Without ignore:  {'_name': 'Entity', '_class_name': 'Entity', '_module_name': 'iglsynth.util.entity', 'coordinate': (10, 20), 'other': 'ignore me'}
With ignore:  {'_name': 'Entity', '_class_name': 'Entity', '_module_name': 'iglsynth.util.entity', 'coordinate': (10, 20)}


## Deriving from Entity: Tips

As `Entity` is expected the superclass of all objects in IGLSynth, developers will want to derive classes from Entity or its derivative. Here are some thumb rules to follow. 

* The constructor of subclass must accept `name` and optional keyword arguments (`**kwargs`). For example, the constructor definition may look like
```python
def __init__(self, name=None, ..., **kwargs):
    ...
```

* **Never** set the `_name` parameter in the subclass. This will corrupt the Entity registry and may result in unexpected behavior of code. 

    Instead, if a new name must be generated for an object using the constructor parameters, then do so before calling the superclass constructor. 
```python
def __init__(self, name, coordinate, **kwargs):
    obj_name = (name, coordinate)
    super().__init__(name=obj_name, **kwargs)
```
